In [1]:
D.<x> = PolynomialRing(QQ)
R.<x> = PolynomialRing(QQ)
S = PowerSeriesRing(QQ,'x', default_prec=300)
L=LaurentSeriesRing(QQ,'x', default_prec=300)
#S.<x,y>=PowerSeriesRing(QQ,default_prec=300)
# Using "R = PolynomialRing(QQ,'x')" produces an error.
# Yet using S as defined here is fine (see definition of Fraleigh4 below)
# But S = PowerSeriesRing(QQ,'x','y') also produces an error.
def pochhammer(x,n):
    return product ([x+k for k in [0..n-1]])
def cRaleigh2(alpha,beta,nu):
    pchhmrA = pochhammer(alpha,nu)
    pchhmrB = pochhammer(beta,nu)
    fnu=factorial(nu)
    answer = D(pchhmrA*pchhmrB/fnu^2)
    return answer
def term(alpha,beta,p):
    return D(1/(alpha + p) + 1/(beta + p) - 2/(1 + p))
def eRaleigh(alpha, beta, nu):
    return D(sum([term(alpha, beta, p) for p in [0..nu - 1]]))
def FstarRaleigh2(alpha,beta,u,terms):
    fsr = sum([cRaleigh2(alpha, beta, nu)*eRaleigh(alpha,beta,nu)*u^nu for nu in [1..terms]])
    return D(fsr)
def Fraleigh2(alpha,beta,u,terms): 
    return D(sum([cRaleigh2(alpha,beta,nu)*u^nu for nu in [0..terms]]))
def FstarRaleigh3(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fsr2 = FstarRaleigh2(alpha,beta,x,n)
    return D(fsr2)
def Fraleigh4(n,m,x):
    alpha = (1/2-1/m)/2
    beta = (1/2+1/m)/2
    fr2 = Fraleigh2(alpha,beta,x,n)
    return D(fr2)
def exNo3c(n,m):
    a1 = S(x)*exp(S(FstarRaleigh3(n,m,x)/Fraleigh4(n,m,x))).O(2*n)
    # S(x) instead of x in df of a1 to avoid an error message
    # after use of the stripQuotationMarks command below.
    a2 = S(a1).O(n+1)
    return a2 # applying S to argument of exp is key.
def J(n,m):
    exn = exNo3c(n+2,m)
    jnm = 1/exn.reverse()
    return jnm.O(n+1) # Jtest is faster than J, and they agree.
def normalJ(n,m):
    f = J(n,m)
    return f.truncate(n+1)
def Jcoefficient(n,m):
    f = J(n+2,m)
    f1 = f.truncate(n+1)
    f1=f1.list()
    f1=Set(f1)
    g1 = f.truncate(n)
    g1 = g1.list()
    g1 = set(g1)
    h = f1.difference(g1)
    h = h.list()
    return h[0]
def xjNew(n,m):
    def xjNewStep(n,m):
        f=J(n,m)
        fl=f.list()
        rt=0
        for i in [0..len(fl)-1]:
            rt=rt+fl[i]*x**i*(m**3*2**6)**i
        return rt
    return xjNewStep(n-1,m)
def j(n,m):return L(xjNew(n,m)/x)
def polynomialCoefficient(n,poly):
    return poly.list()[n]
def leadingCoefficient(polynomial):
    dg=polynomial.degree()
    return polynomialCoefficient(dg,polynomial)
def factorpairToFactor(factorpair):
    (monomial,exponent)=factorpair
    return monomial^exponent
def numericalfactor(polynomial):
    fp=polynomial.factor()
    lst=[factorpairToFactor(factorpair) for factorpair in fp]
    return polynomial/product(lst)
def firstfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.first())
def lastfactor(polynomial):
    fp=polynomial.factor()
    fp=FiniteEnumeratedSet(fp)
    return factorpairToFactor(fp.last())
def reduced(polynomial):
    return numericalfactor(polynomial)*lastfactor(polynomial)
def stripQuotationMarks(string):
    var('x')
    return sage_eval(string,locals={'x':x})
print 89

89


In [3]:
print(j(5,3));print("");print(J(5,3));print normalJ(5,3)

x^-1 + 744 + 196884*x + 21493760*x^2 + 864299970*x^3 + 20245856256*x^4

x^-1 + 31/72 + 1823/27648*x + 10495/2519424*x^2 + 1778395/18345885696*x^3 + 45767/34828517376*x^4 + 41650330075/3327916660110655488*x^5 + O(x^6)
x^-1 + 31/72 + 1823/27648*x + 10495/2519424*x^2 + 1778395/18345885696*x^3 + 45767/34828517376*x^4 + 41650330075/3327916660110655488*x^5


In [2]:
print normalJ(5,3)

x^-1 + 31/72 + 1823/27648*x + 10495/2519424*x^2 + 1778395/18345885696*x^3 + 45767/34828517376*x^4 + 41650330075/3327916660110655488*x^5


In [ ]:
## printed output has been snipped

import pickle
wfile=open('/Users/barrybrent/run1jun21no1.txt','a')
data=[]
for m in [3..500]:
    print m
    data=data+[[m,normalJ(200,m)]]
s=pickle.dumps(str(data))
wfile.write(s)
wfile.close()
#########################
import pickle
rfile = open('/Users/barrybrent/run1jun21no1.txt','r')
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)

polydata=[]

wfile = open('/Users/barrybrent/run14jun21no14.txt','a') # xJ polynomials
for n in [0..200]:
    data=[]
    for k in [0..2*n+1]:
        print (n,k)
        m = s[k][0]
        poly=x*s[k][1] 
        cf=polynomialCoefficient(n,poly)
        data=data+[[m,cf*m^(2*n)]]
    rl=R.lagrange_polynomial(data)
    polydata=polydata+[[n-1,rl]] 

    
t = pickle.dumps(str(polydata))
wfile.write(t)
wfile.close() # print-output snipped

In [3]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
print len(s)
print s[1]

201
[0, 3/8*x^2 + 1/2]


In [4]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print poly

3/8*x^2 + 1/2
69/1024*x^4 - 1/128*x^2 - 3/64
1/128*x^6 - 29/864*x^4 + 1/216*x^2 + 1/54
5601/8388608*x^8 - 3821/524288*x^6 + 5195/262144*x^4 - 101/32768*x^2 - 303/32768
3/65536*x^10 - 16367/18432000*x^8 + 7961/1382400*x^6 - 21809/1728000*x^4 + 373/172800*x^2 + 373/72000
23003/8589934592*x^12 - 2254159/28991029248*x^10 + 338577733/391378894848*x^8 - 106154827/24461180928*x^6 + 68420351/8153726976*x^4 - 4754693/3057647616*x^2 - 4754693/1528823808


In [5]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
bound=12
for k in [1..6]:
    index=s[k][0]
    poly = s[k][1]
    print polynomialnumerator(D(poly))

3*x^2 + 4
69*x^4 - 8*x^2 - 48
27*x^6 - 116*x^4 + 16*x^2 + 64
5601*x^8 - 61136*x^6 + 166240*x^4 - 25856*x^2 - 77568
10125*x^10 - 196404*x^8 + 1273760*x^6 - 2791552*x^4 + 477440*x^2 + 1145856
16769187*x^12 - 486898344*x^10 + 5417243728*x^8 - 27175635712*x^6 + 52546829568*x^4 - 9737611264*x^2 - 19475222528


In [34]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for power in [2..60]:
    flag=1
    for k in [0..len(s)-1]:
        index=s[k][0]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        tf=splitsTF(poly,prime,power)
        if tf==False:flag=-1
    print (power,flag)
    if flag==1:data=data+[power]
print data
    



(2, -1)
(3, -1)
(4, -1)
(5, -1)
(6, -1)
(7, -1)
(8, -1)
(9, -1)
(10, -1)
(11, -1)
(12, 1)
(13, -1)
(14, -1)
(15, -1)
(16, -1)
(17, -1)
(18, -1)
(19, -1)
(20, -1)
(21, -1)
(22, -1)
(23, -1)
(24, 1)
(25, -1)
(26, -1)
(27, -1)
(28, -1)
(29, -1)
(30, -1)
(31, -1)
(32, -1)
(33, -1)
(34, -1)
(35, -1)
(36, 1)
(37, -1)
(38, -1)
(39, -1)
(40, -1)
(41, -1)
(42, -1)
(43, -1)
(44, -1)
(45, -1)
(46, -1)
(47, -1)
(48, 1)
(49, -1)
(50, -1)
(51, -1)
(52, -1)
(53, -1)
(54, -1)
(55, -1)
(56, -1)
(57, -1)
(58, -1)
(59, -1)
(60, 1)
[12, 24, 36, 48, 60]


In [35]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=3
bound=120
data=[]
for j in [1..24]:
    flag=1
    for k in [0..len(s)-1]:
        index=s[k][0]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        tf=splitsTF(poly,prime,12*j)
        if tf==False:flag=-1
    print (j,flag)
    if flag==1:data=data+[j]
print data

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [36]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
bound=120
data=[]
for power in [2..60]:
    flag=1
    for k in [0..len(s)-1]:
        index=s[k][0]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        tf=splitsTF(poly,prime,power)
        if tf==False:flag=-1
    print (power,flag)
    if flag==1:data=data+[power]
print data
    

(2, -1)
(3, -1)
(4, -1)
(5, -1)
(6, -1)
(7, -1)
(8, -1)
(9, -1)
(10, -1)
(11, -1)
(12, 1)
(13, -1)
(14, -1)
(15, -1)
(16, -1)
(17, -1)
(18, -1)
(19, -1)
(20, -1)
(21, -1)
(22, -1)
(23, -1)
(24, 1)
(25, -1)
(26, -1)
(27, -1)
(28, -1)
(29, -1)
(30, -1)
(31, -1)
(32, -1)
(33, -1)
(34, -1)
(35, -1)
(36, 1)
(37, -1)
(38, -1)
(39, -1)
(40, -1)
(41, -1)
(42, -1)
(43, -1)
(44, -1)
(45, -1)
(46, -1)
(47, -1)
(48, 1)
(49, -1)
(50, -1)
(51, -1)
(52, -1)
(53, -1)
(54, -1)
(55, -1)
(56, -1)
(57, -1)
(58, -1)
(59, -1)
(60, 1)
[12, 24, 36, 48, 60]


In [37]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=5
bound=120
data=[]
for j in [1..24]:
    flag=1
    for k in [0..len(s)-1]:
        index=s[k][0]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        tf=splitsTF(poly,prime,12*j)
        if tf==False:flag=-1
    print (j,flag)
    if flag==1:data=data+[j]
print data

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
(7, 1)
(8, 1)
(9, 1)
(10, 1)
(11, 1)
(12, 1)
(13, 1)
(14, 1)
(15, 1)
(16, 1)
(17, 1)
(18, 1)
(19, 1)
(20, 1)
(21, 1)
(22, 1)
(23, 1)
(24, 1)
[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24]


In [30]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
bound=120
data=[]
for power in [2..60]:
    flag=1
    for k in [0..len(s)-1]:
        index=s[k][0]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        tf=splitsTF(poly,prime,power)
        if tf==False:flag=-1
    print (power,flag)
    if flag==1:data=data+[power]
print data
    


(2, -1)
(3, -1)
(4, -1)
(5, -1)
(6, -1)
(7, -1)
(8, -1)
(9, -1)
(10, -1)
(11, -1)
(12, -1)
(13, -1)
(14, -1)
(15, -1)
(16, -1)
(17, -1)
(18, -1)
(19, -1)
(20, -1)
(21, -1)
(22, -1)
(23, -1)
(24, -1)
(25, -1)
(26, -1)
(27, -1)
(28, -1)
(29, -1)
(30, -1)
(31, -1)
(32, -1)
(33, -1)
(34, -1)
(35, -1)
(36, -1)
(37, -1)
(38, -1)
(39, -1)
(40, -1)
(41, -1)
(42, -1)
(43, -1)
(44, -1)
(45, -1)
(46, -1)
(47, -1)
(48, -1)
(49, -1)
(50, -1)
(51, -1)
(52, -1)
(53, -1)
(54, -1)
(55, -1)
(56, -1)
(57, -1)
(58, -1)
(59, -1)
(60, 1)
[60]


In [33]:
import pickle
rfile = open('/Users/barrybrent/run14jun21no14.txt','r') # xJ polynomials
rs=pickle.load(rfile)
rfile.close()
s=stripQuotationMarks(rs)
prime=7
bound=120
data=[]
for j in [1..6]:
    flag=1
    for k in [0..len(s)-1]:
        index=s[k][0]
        poly = s[k][1]
        poly=polynomialnumerator(D(poly))
        tf=splitsTF(poly,prime,60*j)
        if tf==False:flag=-1
    print (j,flag)
    if flag==1:data=data+[j]
print data

(1, 1)
(2, 1)
(3, 1)
(4, 1)
(5, 1)
(6, 1)
[1, 2, 3, 4, 5, 6]
